In [1]:
import sqlalchemy

#engine = sqlalchemy.create_engine('postgresql://localhost:5433', echo = False)

engine = sqlalchemy.create_engine("postgresql://@:5433/nobodyxu")
engine.execute("""
    DROP TABLE IF EXISTS covid19_cases
""")

In [2]:
import pandas as pds
import urllib

url = 'https://data.nsw.gov.au/data/api/3/action/datastore_search?offset={}&resource_id=21304414-1ff1-4243-a5d2-f52778048b29'

offset = 0
total = 1

# Assumption: The data can only grow
while offset < total:
    raw_df = pds.read_json(urllib.request.urlopen(url.format(offset)))
    
    df = pds.DataFrame(raw_df.loc["records", "result"])
    print(df)
    df.to_sql('covid19_cases', con = engine, if_exists = 'append', index = False)
    
    offset += 100
    total = raw_df.loc["total"]["result"]

   notification_date  postcode lhd_2010_code          lhd_2010_name  \
0         2020-01-22      2134          X700                 Sydney   
1         2020-01-24      2121          X760        Northern Sydney   
2         2020-01-25      2071          X760        Northern Sydney   
3         2020-01-25      2033          X720   South Eastern Sydney   
4         2020-01-30      2350          X800     Hunter New England   
..               ...       ...           ...                    ...   
95        2020-03-11      2766          X740         Western Sydney   
96        2020-03-11      2766          X740         Western Sydney   
97        2020-03-11      2020          X720   South Eastern Sydney   
98        2020-03-11      2114          X760        Northern Sydney   
99        2020-03-12      2756          X750  Nepean Blue Mountains   

    lga_code19             lga_name19  
0      11300.0            Burwood (A)  
1      16260.0         Parramatta (C)  
2      14500.0        Ku-ri

In [3]:
engine.execute("""
    SELECT *
      FROM covid19_cases
     LIMIT 25;
""").fetchall()

[('2020-01-22', 2134, 'X700', 'Sydney', 11300.0, 'Burwood (A)'),
 ('2020-01-24', 2121, 'X760', 'Northern Sydney', 16260.0, 'Parramatta (C)'),
 ('2020-01-25', 2071, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 ('2020-01-25', 2033, 'X720', 'South Eastern Sydney', 16550.0, 'Randwick (C)'),
 ('2020-01-30', 2350, 'X800', 'Hunter New England', 10130.0, 'Armidale Regional (A)'),
 ('2020-02-21', 2010, 'X720', 'South Eastern Sydney', 17200.0, 'Sydney (C)'),
 ('2020-02-26', 2073, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 ('2020-02-27', 2070, 'X760', 'Northern Sydney', 14500.0, 'Ku-ring-gai (A)'),
 ('2020-02-28', 2750, 'X750', 'Nepean Blue Mountains', 16350.0, 'Penrith (C)'),
 ('2020-02-29', 2077, 'X760', 'Northern Sydney', 14000.0, 'Hornsby (A)'),
 ('2020-03-01', 2163, 'X710', 'South Western Sydney', 12850.0, 'Fairfield (C)'),
 ('2020-03-01', 2022, 'X720', 'South Eastern Sydney', 18050.0, 'Waverley (A)'),
 ('2020-03-01', 2163, 'X710', 'South Western Sydney', 12850.0, '

In [4]:
! /usr/lib/postgresql/12/bin/pg_dump -p 5433 -f covid19_cases/ --format=d --create --jobs="$(nproc)" -Z 0 -v --table=covid19_cases

pg_dump: last built-in OID is 16383
pg_dump: reading extensions
pg_dump: identifying extension members
pg_dump: reading schemas
pg_dump: reading user-defined tables
pg_dump: reading user-defined functions
pg_dump: reading user-defined types
pg_dump: reading procedural languages
pg_dump: reading user-defined aggregate functions
pg_dump: reading user-defined operators
pg_dump: reading user-defined access methods
pg_dump: reading user-defined operator classes
pg_dump: reading user-defined operator families
pg_dump: reading user-defined text search parsers
pg_dump: reading user-defined text search templates
pg_dump: reading user-defined text search dictionaries
pg_dump: reading user-defined text search configurations
pg_dump: reading user-defined foreign-data wrappers
pg_dump: reading user-defined foreign servers
pg_dump: reading default privileges
pg_dump: reading user-defined collations
pg_dump: reading user-defined conversions
pg_dump: reading type casts
pg_dump: reading transforms
pg_d